In [3]:
%%time
import zero_shot
from importlib import reload
zero_shot = reload(zero_shot)
import eval
import pandas as pd

CPU times: user 4.57 ms, sys: 0 ns, total: 4.57 ms
Wall time: 3.81 ms


In [4]:
%%time
model_paths = ['./checkpoints/pt-imp/checkpoint_11900_LIT_CXBERT.pt']
cxr_filepath = '/home/ec2-user/cs197-final-project/data/chexpert_test.h5'
cxr_labels = pd.read_csv("./data/CheXpert/test_labels.csv").columns.tolist()[2:]
cxr_pair_template = ("{}", "no {}")
cache_dir = None

predictions, y_pred_avg = zero_shot.ensemble_models(
    model_paths=model_paths, 
    cxr_filepath=cxr_filepath, 
    cxr_labels=cxr_labels, 
    cxr_pair_template=cxr_pair_template, 
    cache_dir=cache_dir,
)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/opt/conda/envs/cs197-env/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


Inferring model ./checkpoints/pt-imp/checkpoint_11900_LIT_CXBERT.pt


  0%|          | 0/13 [00:00<?, ?it/s]

AttributeError: 'Tensor' object has no attribute 'input_ids'

In [14]:
cxr_true_labels_path = "../cheXpert-test-set-labels/groundtruth.csv"

# loads in ground truth labels into memory
test_pred = y_pred_avg
test_true = zero_shot.make_true_labels(cxr_true_labels_path=cxr_true_labels_path, cxr_labels=cxr_labels)

# evaluate model, no bootstrap
cxr_results: pd.DataFrame = eval.evaluate(test_pred, test_true, cxr_labels) # eval on full test datset

# boostrap evaluations for 95% confidence intervals
bootstrap_results = eval.bootstrap(test_pred, test_true, cxr_labels) # (df of results for each bootstrap, df of CI)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [15]:
# print results with confidence intervals
bootstrap_results[1]

,Enlarged Cardiomediastinum_auc,Cardiomegaly_auc,Lung Opacity_auc,Lung Lesion_auc,Edema_auc,Consolidation_auc,Pneumonia_auc,Atelectasis_auc,Pneumothorax_auc,Pleural Effusion_auc,Pleural Other_auc,Fracture_auc,Support Devices_auc
mean,0.8987,0.8982,0.8989,0.6512,0.8904,0.8838,0.7111,0.7627,0.6421,0.9165,0.7100,0.5718,0.5434
lower,0.8721,0.8667,0.8691,0.4347,0.8546,0.8149,0.4983,0.7204,0.4642,0.8866,0.5291,0.1476,0.4910
upper,0.9233,0.9233,0.9225,0.8579,0.9228,0.9396,0.8804,0.8039,0.8150,0.9404,0.9719,0.9506,0.5906


In [16]:
cxr_filepath_padchest = './data/padchest_sample.h5'

predictions_padchest, y_pred_avg_padchest = zero_shot.ensemble_models(
    model_paths=model_paths, 
    cxr_filepath=cxr_filepath_padchest, 
    cxr_labels=cxr_labels, 
    cxr_pair_template=cxr_pair_template, 
    cache_dir=cache_dir,
)

Inferring model ./checkpoints/chexzero_weights/best_64_0.0001_original_16000_0.861.pt


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [19]:
pd.DataFrame(data=y_pred_avg_padchest, columns=cxr_labels)

,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,0.488066,0.480624,0.494400,0.498611,0.491019,0.498194,0.501676,0.491301,0.501997,0.497201,0.498265,0.499409,0.500155
1,0.484796,0.479073,0.492584,0.495209,0.490724,0.494591,0.498831,0.492438,0.498729,0.490771,0.495506,0.498938,0.499630
2,0.487437,0.483478,0.491547,0.495870,0.487901,0.494389,0.498094,0.490071,0.500478,0.490752,0.495742,0.498848,0.499900
3,0.484334,0.476894,0.490967,0.494940,0.491341,0.493689,0.500145,0.488874,0.498865,0.488994,0.494716,0.499725,0.499320
4,0.494663,0.495300,0.494119,0.500408,0.492729,0.495521,0.499312,0.495132,0.497716,0.494433,0.497239,0.501378,0.500160
5,0.503762,0.501488,0.502128,0.503772,0.496702,0.500623,0.501657,0.499967,0.499954,0.500284,0.499450,0.498832,0.498958
6,0.491341,0.487890,0.492717,0.497709,0.488972,0.494383,0.497461,0.493130,0.498469,0.491743,0.496321,0.501677,0.498972
7,0.488911,0.485779,0.496029,0.497645,0.495411,0.496222,0.501150,0.491911,0.502172,0.493976,0.497800,0.499405,0.500446
8,0.503159,0.501851,0.502814,0.506068,0.498228,0.502078,0.500297,0.504505,0.498954,0.507012,0.503187,0.500734,0.500481
9,0.502624,0.496496,0.504250,0.503030,0.498814,0.501142,0.502928,0.502645,0.496986,0.497470,0.498450,0.500390,0.498671
